In [1]:
import pandas as pd
import numpy as np

# === Tahap 1: Load Data ===
def load_data():
    data_optimasi = pd.read_csv('C:/Users/User/Documents/SKRIPSI/OPTIMASI PER PENYAKIT/Start_10,10/hasil_optimasi.csv')
    data_rekam = pd.read_csv('C:/Users/User/Documents/SKRIPSI/OPTIMASI PER PENYAKIT/Start_10,10/data_rekam_medis.csv')
    return data_optimasi, data_rekam

# === Tahap 2: Fungsi Kombinasi CF ===
def kombinasi_cf(cf_list):
    if not cf_list:
        return 0
    combined = cf_list[0]
    for cf in cf_list[1:]:
        combined = combined + cf * (1 - combined)
    return combined

# === Tahap 3: Buat Dictionary CF ===
def buat_lookup_cf(data_optimasi):
    cf_pakar = {(row['kode_penyakit'], row['kode_gejala']): row['cf_pakar']
                for _, row in data_optimasi.iterrows()}
    cf_optimasi = {(row['kode_penyakit'], row['kode_gejala']): row['cf_optimasi']
              for _, row in data_optimasi.iterrows()}
    return cf_pakar, cf_optimasi

# === Tahap 4: Hitung CF per pasien ===
def hitung_cf_pasien(group, penyakit_list, cf_dict):
    gejala_pasien = group['kode_gejala'].tolist()
    cf_user_pasien = group['cf_user'].tolist()

    hasil_cf = {}
    for penyakit in penyakit_list:
        cf_list = []
        for gejala, cf_user in zip(gejala_pasien, cf_user_pasien):
            nilai_cf = cf_dict.get((penyakit, gejala))
            if nilai_cf is not None:
                cf_list.append(cf_user * nilai_cf)
        hasil_cf[penyakit] = kombinasi_cf(cf_list)
    return hasil_cf

# === Tahap 5: Proses Utama ===
def proses_diagnosis(data_optimasi, data_rekam):
    cf_pakar_dict, cf_optimasi_dict = buat_lookup_cf(data_optimasi)
    penyakit_list = data_optimasi['kode_penyakit'].unique()

    hasil = []

    for kode_pasien, group in data_rekam.groupby('kode_pasien'):
        diagnosis_asli = group['diagnosis'].iloc[0]

        cf_sebelum = hitung_cf_pasien(group, penyakit_list, cf_pakar_dict)
        cf_setelah = hitung_cf_pasien(group, penyakit_list, cf_optimasi_dict)

        diag_sebelum = max(cf_sebelum.items(), key=lambda x: x[1])
        diag_setelah = max(cf_setelah.items(), key=lambda x: x[1])

        hasil.append({
            'kode_pasien': kode_pasien,
            'diagnosis_asli': diagnosis_asli,
            'diagnosis_sebelum': diag_sebelum[0],
            'cf_sebelum': diag_sebelum[1],
            'cocok_sebelum': diag_sebelum[0] == diagnosis_asli,
            'diagnosis_setelah': diag_setelah[0],
            'cf_setelah': diag_setelah[1],
            'cocok_setelah': diag_setelah[0] == diagnosis_asli
        })

    return pd.DataFrame(hasil)

        # === Tahap 6: Evaluasi Hasil CF ===
def evaluasi_cf(cf_sebelum, cf_setelah):
    if cf_setelah > cf_sebelum:
        return 'lebih_unggul'
    elif cf_setelah == cf_sebelum:
        return 'sama'
    else:
        return 'belum_teroptimasi'

def evaluasi_hasil(hasil_df):
    hasil_df['cf_lebih_unggul'] = hasil_df.apply(
        lambda row: evaluasi_cf(row['cf_sebelum'], row['cf_setelah']),
        axis=1
    )
    return hasil_df




# === Main ===
if __name__ == "__main__":
    data_optimasi, data_rekam = load_data()
    hasil_df = proses_diagnosis(data_optimasi, data_rekam)
    hasil_df = evaluasi_hasil(hasil_df)

    print(hasil_df)
    hasil_df.to_csv('hasil_perbandingan_diagnosis_FIX.csv', index=False)


   kode_pasien diagnosis_asli diagnosis_sebelum  cf_sebelum  cocok_sebelum  \
0        PAS01            P03               P06    0.978227          False   
1        PAS02            P03               P07    0.951040          False   
2        PAS03            P02               P02    0.962560           True   
3        PAS04            P04               P04    0.891136           True   
4        PAS05            P06               P07    0.990208          False   
..         ...            ...               ...         ...            ...   
95       PAS95            P03               P06    0.966400          False   
96       PAS96            P06               P06    0.985600           True   
97       PAS97            P06               P06    0.997120           True   
98       PAS98            P04               P04    0.928000           True   
99       PAS99            P06               P06    0.987904           True   

   diagnosis_setelah  cf_setelah  cocok_setelah    cf_lebih_ung

In [2]:
# Statistik cocok_sebelum dan cocok_setelah

# Hitung total pasien
total_pasien = len(hasil_df)

# Hitung pasien cocok sebelum dan sesudah
jumlah_cocok_sebelum = hasil_df['cocok_sebelum'].sum()
jumlah_cocok_setelah = hasil_df['cocok_setelah'].sum()

# Hitung persentasenya
persen_cocok_sebelum = (jumlah_cocok_sebelum / total_pasien) * 100
persen_cocok_setelah = (jumlah_cocok_setelah / total_pasien) * 100

# Tampilkan hasil
print("Total Pasien:", total_pasien)
print("cocok Sebelum Optimasi: {} pasien ({:.2f}%)".format(jumlah_cocok_sebelum, persen_cocok_sebelum))
print("cocok Sesudah Optimasi: {} pasien ({:.2f}%)".format(jumlah_cocok_setelah, persen_cocok_setelah))


Total Pasien: 100
cocok Sebelum Optimasi: 85 pasien (85.00%)
cocok Sesudah Optimasi: 88 pasien (88.00%)


In [3]:
# Pasien yang cocok baik sebelum maupun sesudah optimasi
mask_sama_cocok = (hasil_df['cocok_sebelum'] == True) & (hasil_df['cocok_setelah'] == True)
jumlah_sama_cocok = mask_sama_cocok.sum()

print("Jumlah pasien dengan diagnosis yang cocok di SEBELUM dan SESUDAH optimasi:", jumlah_sama_cocok)


Jumlah pasien dengan diagnosis yang cocok di SEBELUM dan SESUDAH optimasi: 82


In [4]:
# Filter hanya pasien yang diagnosisnya cocok sebelum dan sesudah
df_sama_cocok = hasil_df[mask_sama_cocok]

# Hitung statistik evaluasi nilai CF
jumlah_lebih_unggul = (df_sama_cocok['cf_lebih_unggul'] == 'lebih_unggul').sum()
jumlah_sama = (df_sama_cocok['cf_lebih_unggul'] == 'sama').sum()
jumlah_belum = (df_sama_cocok['cf_lebih_unggul'] == 'belum_teroptimasi').sum()

print("Total pasien dengan diagnosis yang cocok di sebelum dan sesudah optimasi:", len(df_sama_cocok))
print("Pasien dengan nilai CF lebih unggul setelah optimasi:", jumlah_lebih_unggul)
print("Pasien dengan nilai CF tetap:", jumlah_sama)
print("Pasien yang belum teroptimasi:", jumlah_belum)




Total pasien dengan diagnosis yang cocok di sebelum dan sesudah optimasi: 82
Pasien dengan nilai CF lebih unggul setelah optimasi: 58
Pasien dengan nilai CF tetap: 2
Pasien yang belum teroptimasi: 22


In [5]:
total_pasien = len(hasil_df)
persen_cf_naik = (jumlah_lebih_unggul / jumlah_sama_cocok) * 100

print("Persentase pasien dengan kenaikan nilai CF dari pasien yang sama-sama cocok :", round(persen_cf_naik, 2), "%")


Persentase pasien dengan kenaikan nilai CF dari pasien yang sama-sama cocok : 70.73 %
